## BME i9400
## Fall 2025

### Homework 1: Learning Periodic Structure Beyond Polynomials

**Assigned:** 2025-10-06  
**Due:** 2025-10-20 11:59:59.999 PM EST

**Place completed notebook into your "my-work" folder on JupyterHub**

**Learning goals**
- Understand **why** polynomial regression fails on multi‑cycle periodic data.
- Implement **your own cross‑validation loop** using provided k‑fold indices.
- Build sinusoidal/Fourier feature bases and use **ridge regression** with CV.

**Honor & AI use policy (read carefully)**
- You may use docs/StackOverflow for syntax.  
- You **may** ask an LLM for debugging/snippets, but you must include an **AI Log** at the end (prompts + what you used).
- You **may not** ask for a full solution. Your code and plots must reflect your own understanding.

**Deliverables**
1. Executed notebook (.ipynb) with all cells run. 
2. `ai_log.md` (if you used an LLM).

**What YOU write (student-supplied code):**
- `poly_features`
- The **cross‑validation loop** for polynomials (store per‑degree train/val MSEs)
- `fourier_features` (no trend; bias optional)
- Parts of `cv_ridge` (append fold MSEs; record means; choose best α)

**Provided (do not change):** helpers (`seed_from_string`, `kfold_indices`, `mse`, `fit_ols`, `predict`), plotting scaffolds, and data generator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['figure.dpi'] = 120

def seed_from_string(s: str) -> int:
    import hashlib
    h = hashlib.sha256(s.encode('utf-8')).digest()
    return int.from_bytes(h[:4], 'little', signed=False)

def train_test_split(X, y, train_frac=0.8, shuffle=True, seed=0):
    n = len(y)
    idx = np.arange(n)
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(idx)
    cut = int(train_frac*n)
    tr, te = idx[:cut], idx[cut:]
    return X[tr], X[te], y[tr], y[te]

def mse(y_true, y_pred):
    return float(np.mean((y_true - y_pred)**2))

def kfold_indices(n, k=5, seed=0):
    idx = np.arange(n)
    rng = np.random.default_rng(seed)
    rng.shuffle(idx)
    folds = np.array_split(idx, k)
    return folds

def fit_ols(X: np.ndarray, y: np.ndarray) -> np.ndarray:
    # Closed-form least squares using pinv for stability.
    return np.linalg.pinv(X) @ y

def predict(X: np.ndarray, w: np.ndarray) -> np.ndarray:
    return X @ w

## Part 1 — Create your **multi‑cycle** dataset (unique per student)

We model

$y(t) = \alpha_0 + \alpha_1 t + A \sin(\omega t + \phi) + \epsilon,\quad \epsilon\sim \mathcal{N}(0,\sigma^2). $

Target **4–8 cycles**. Use your CUNY email or EMPLID as the RNG seed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_dataset(student_key: str,
                     n_points: int = 400,
                     t_span: Tuple[float, float] = (0.0, 40.0),
                     min_cycles: float = 4.0,
                     max_cycles: float = 8.0,
                     noise_sigma: float = 0.15,
                     trend_mag: float = 0.03):
    seed = seed_from_string(student_key)
    rng = np.random.default_rng(seed)
    t0, t1 = t_span
    t = np.linspace(t0, t1, n_points)
    total_cycles = rng.uniform(min_cycles, max_cycles)
    omega = 2*np.pi*total_cycles/(t1 - t0)
    A = rng.uniform(0.8, 1.2)
    phi = rng.uniform(-np.pi, np.pi)
    alpha0 = rng.uniform(-0.5, 0.5)
    alpha1 = rng.uniform(-trend_mag, trend_mag)
    eps = rng.normal(0.0, noise_sigma, size=n_points)
    y = alpha0 + alpha1*t + A*np.sin(omega*t + phi) + eps
    truth = dict(A=A, omega=omega, phi=phi, alpha0=alpha0, alpha1=alpha1, sigma=noise_sigma)
    return t, y, truth

# TODO: Replace with your CUNY email/EMPLID for a unique dataset
STUDENT_KEY = "firstname.lastname@cuny.edu"

t, y, truth = generate_dataset(STUDENT_KEY)
print("truth (shown for learning; grading will use hidden seeds):", truth)

plt.plot(t, y, '.', label='observed')
plt.xlabel("t"); plt.ylabel("y"); plt.legend(); plt.title("Your unique dataset"); plt.show()

### What is K-fold Cross-Validation (CV)?

**Goal.** Use all of the data without training and testing on overlapping samples. 
Cross-validation is used to select the best model from a set of candidates. 

**K-fold CV.** 
- Randomly partition indices into $K$ folds (equal sized chunks).
- For each fold $j$: train the model on all examples except for the ones in fold $j$.
- Take the trained model and apply it to the examples in fold $j$. Store the performance and repeat for all $K$ folds. 

**In this HW.**
- Use `kfold_indices(n, k=5, seed=...)` once to get folds.
- For each candidate model: loop folds → fit on $K-1$ folds, score on the held-out fold, **store** MSEs → average them.
- Pick the candidate model with the **lowest** mean validation MSE.


## Part 2 — Baseline: polynomial regression & **your** CV loop

Implement `poly_features` and compose a k‑fold CV loop (not a helper function).
- Degrees: 1..12
- Use `kfold_indices(n, k=5, seed=...)` to make folds
- For each degree, compute **train** and **validation** MSE and store them

In [ ]:
import numpy as np

# === STUDENT TODO (implement this function) ===
def poly_features(x: np.ndarray, degree: int, include_bias=True) -> np.ndarray:
    # Return design matrix [1, x, x^2, ..., x^degree] if include_bias=True,
    # else [x, x^2, ..., x^degree].
    # TODO(Student): implement and return an array of shape (n_samples, degree+include_bias)
    raise NotImplementedError("Implement poly_features")

degrees = list(range(1, 13))
cv_seed = seed_from_string(STUDENT_KEY) ^ 0xABCDEF
k = 5

train_mse_per_deg = []
val_mse_per_deg = []

# === STUDENT TODO: Compose the CV loop using kfold_indices ===
# Outline:
# 1) Build X for the current degree
# 2) Get folds = kfold_indices(len(t), k=k, seed=cv_seed)
# 3) For each fold, fit on union of others (fit_ols), score on that fold
# 4) Store mean train MSE and mean val MSE for this degree
for d in degrees:
    # TODO(Student): build X for degree d
    # TODO(Student): get folds
    # TODO(Student): loop folds, split indices, fit and compute MSEs
    # TODO(Student): append the mean train/val MSE to the lists
    raise NotImplementedError("Compose the CV loop and record MSEs")

# === Plotting (leave as-is) ===
best_degree_idx = int(np.argmin(val_mse_per_deg))
best_degree = degrees[best_degree_idx]
print("Best degree by CV:", best_degree)

# Fit a model at the selected degree on all data (for visualization)
X_full = poly_features(t, best_degree, include_bias=True)
w_full = fit_ols(X_full, y)
yhat = predict(X_full, w_full)

import matplotlib.pyplot as plt
plt.plot(t, y, '.', label='observed')
plt.plot(t, yhat, '-', label=f'poly deg={best_degree}')
plt.xlabel("t"); plt.ylabel("y"); plt.legend(); plt.title("Polynomial fit")
plt.show()

plt.plot(degrees, train_mse_per_deg, '-o', label='train MSE')
plt.plot(degrees, val_mse_per_deg, '-o', label='CV MSE')
plt.xlabel("degree"); plt.ylabel("MSE"); plt.legend(); plt.title("Polynomial selection by CV")
plt.show()

print("Discussion prompt: why do polynomials fail to capture multi-cycle periodicity here?")

## Part 3 — Fourier features with **ridge** (parts supplied by you)

You will write:
- `fourier_features` (no linear trend; bias optional only)
- Parts of `cv_ridge` to accumulate fold MSEs and choose the best α


How to build the sinusoidal (Fourier) design matrix:
    - Rows = samples. Each row corresponds to one time point in your data (e.g., t[i]).
    - Columns = features. We’ll use a bias column (all ones) plus pairs of sine/cosine columns for each candidate frequency in your grid.

Column order (must be exactly this):

[ 1 , cos(ω0 * t) , sin(ω0 * t) , cos(ω1 * t) , sin(ω1 * t) , ... ]

    - If you have n samples and m frequencies, the matrix shape is n x (1 + 2*m).
    - We keep the bias in column 0 on purpose, so later we can easily find the cosine/sine weights for frequency j at positions offset + 2*j and offset + 2*j + 1 with offset = 1.


Implementation tips:
    - Start a list of columns with the bias: cols = [np.ones_like(t).reshape(-1,1)].
    - For each frequency w in your grid:
        - compute np.cos(w*t) and np.sin(w*t),
        - reshape to column vectors (e.g., .reshape(-1,1)),
        - append them to cols.
    - Finish with X = np.hstack(cols).


In [ ]:
import numpy as np

# === STUDENT TODO (implement this function) ===
def fourier_features(t: np.ndarray, omegas: np.ndarray, include_bias: bool = True) -> np.ndarray:
    # Build columns: [1] (optional), then for each ω_j in omegas: [cos(ω_j t), sin(ω_j t)].
    # TODO(Student): implement; shape should be (n_samples, (1 if bias else 0) + 2*len(omegas))
    raise NotImplementedError("Implement fourier_features")

def fit_ridge(X: np.ndarray, y: np.ndarray, alpha: float) -> np.ndarray:
    XT_X = X.T @ X
    I = np.eye(X.shape[1])
    I[0, 0] = 0.0  # do not penalize bias
    A = XT_X + alpha*I
    return np.linalg.pinv(A) @ (X.T @ y)

def cv_ridge(X: np.ndarray, y: np.ndarray, alphas: List[float], k=5, seed=0):
    folds = kfold_indices(len(y), k=k, seed=seed)
    mean_mse_per_alpha = []
    for a in alphas:
        fold_mses = []
        for i in range(k):
            va_idx = folds[i]
            tr_idx = np.concatenate([folds[j] for j in range(k) if j != i])
            w = fit_ridge(X[tr_idx], y[tr_idx], alpha=a)
            # === STUDENT TODO: compute validation MSE for this fold and append ===
            # Replace the next line with the correct MSE computation
            fold_mses.append(0.0)
        # === STUDENT TODO: append the mean of fold_mses to mean_mse_per_alpha ===
        mean_mse_per_alpha.append(0.0)
    # === STUDENT TODO: choose best_alpha (argmin over mean_mse_per_alpha) ===
    best_alpha = float(alphas[0])
    return best_alpha, mean_mse_per_alpha

# Build frequency grid and run
span = float(t[-1] - t[0])
omega_grid = 2*np.pi*np.linspace(2.0, 14.0, 120)/span

Xf = fourier_features(t, omega_grid, include_bias=True)
alphas = np.logspace(-6, 2, 16)

best_alpha, alpha_curve = cv_ridge(Xf, y, alphas, k=5, seed=seed_from_string(STUDENT_KEY)^0x55AA)
print("Best alpha:", best_alpha)

wf = fit_ridge(Xf, y, alpha=best_alpha)
yhat_f = Xf @ wf

import matplotlib.pyplot as plt
plt.plot(t, y, '.', label='observed')
plt.plot(t, yhat_f, '-', label='Fourier+ridge')
plt.xlabel("t"); plt.ylabel("y"); plt.legend(); plt.title("Fourier basis reconstruction")
plt.show()

# Rank frequencies by amplitude sqrt(cos^2 + sin^2)
offset = 1  # bias only (no trend)
amps = []
for j, w in enumerate(omega_grid):
    c = wf[offset + 2*j]      # cos coeff
    s = wf[offset + 2*j + 1]  # sin coeff
    amps.append((float(np.sqrt(c*c + s*s)), float(w)))
amps_sorted = sorted(amps, key=lambda x: -x[0])
top3 = amps_sorted[:3]
print("Top-3 frequency estimates (amplitude, omega):", top3)

print("Answer prompt: Compare your top-3 ω to the true ω in `truth`. Discuss grid resolution and ridge effects.")

## Appendix — AI Log (required **only** if you used an LLM)
Paste your prompts and briefly note what you used them for.